## Experiment

## Experiement Design

### Metric Choice

To select appropriate metrics for our experiment. A funnel diagram shown below is useful:

View the course overview page

              |

Click the “Start free trial button”

              |

Complete checkout

              |

Remain enrolled past the 14-day boundary

Based on the funnel analysis we have the following choice of metrics:

#### Invariant metric
- Number of cookies: That is, number of unique cookies to view the course overview page. (dmin=3000)
- Number of clicks: That is, number of unique cookies to click the "Start free trial" button (which happens before the free trial screener is trigger). (dmin=240)
- Click-through-probability: That is, number of unique cookies to click the "Start free trial" button divided by number of unique cookies to view the course overview page. (dmin=0.01)
#### Evaluation metric
- Gross conversion: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. (dmin= 0.01)
- Retention: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout. (dmin=0.01)
- Net conversion: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button. (dmin= 0.0075)

For all three evaluation metrics, we expect to see them increase, if the tested change does make a difference.

### Measuring Standard Deviation

Since each of our evaluation metric is a probability, we can assume their sampling distributions follow Binomial distributions. We estimate their standard deviation for a sample size of 5000 cookies by using the given baseline data. 

In [2]:
import numpy as np
import pandas as pd

In [3]:
baseline = pd.read_csv("BaselineValues.csv", header=None)
baseline

,0,1
0,Unique cookies to view course overview page pe...,40000.000000
1,"Unique cookies to click ""Start free trial"" per...",3200.000000
2,Enrollments per day:,660.000000
3,"Click-through-probability on ""Start free trial"":",0.080000
4,"Probability of enrolling, given click:",0.206250
5,"Probability of payment, given enroll:",0.530000
6,"Probability of payment, given click",0.109313


The standard deviation of a Binomial distribution is given by sqrt(p(1-p)/N). The tricky part is to calculate the correct N for each evalution metric corresponding to 5000 cookies

In [4]:
# read the table
n = baseline.iloc[0,1]
n_click = baseline.iloc[1,1]
n_enroll = baseline.iloc[2,1]
p_gc = baseline.iloc[4,1]
p_rt = baseline.iloc[5,1]
p_nc = baseline.iloc[6,1]
sample_size = 5000

In [5]:
std_gc = np.sqrt(p_gc*(1-p_gc) / ((n_click/n)*sample_size))
std_nc = np.sqrt(p_nc*(1-p_nc) / ((n_click/n)*sample_size))
std_rt = np.sqrt(p_rt*(1-p_rt) / ((n_enroll/n)*sample_size))

digits = 4
print("STD of gross conversion probability: ", round(std_gc, digits))
print("STD of net conversion probability: ", round(std_nc, digits))
print("STD of retention probability: ", round(std_rt, digits))

STD of gross conversion probability:  0.0202
STD of net conversion probability:  0.0156
STD of retention probability:  0.0549


Since all of our evaluation metrics are probabilities, it's probably safe to calulate their std analytically. 

### Sizing

#### Choosing number of samples given power

In [6]:
from scipy.stats import norm
def get_beta(N, alpha, dmin, s0):
    std = s0 * np.sqrt(2/N)
    z = norm.ppf(1-alpha/2)
    beta = norm.cdf(z*std, dmin, std) - norm.cdf(-z*std, dmin, std)
    return beta

def test_size(alpha, beta, dmin, s0, N0=1):
    """
    alpha:probability of rejecting the null when null is true
    beta: probability of accepting the null when null is false
    dmin: practical significance boundary
    s0: standard deviation when N=1
    N0: start searching size
    
    return: total number of samples required per group
    This method is obviously too slow if the required size is very large. Need to use implemented inverse function to solve N faster.
    """
    N = N0
    while get_beta(N, alpha, dmin, s0) >beta:
        N += 1
    return N

In [7]:
# we impose an overall alpha = 0.05 by using the Bonferroni correction
alpha = 0.05/3
# we require beta = 0.2 for each metric
# the result of each metric needs to be converted to the total number of cookies to be collected
N_gc = test_size(alpha, 0.2, 0.01, np.sqrt(p_gc*(1-p_gc)), N0=n_click) / (n_click/n)
N_nc = test_size(alpha, 0.2, 0.0075, np.sqrt(p_nc*(1-p_nc)), N0=n_click) / (n_click/n)
N_rt = test_size(alpha, 0.2, 0.01, np.sqrt(p_rt*(1-p_rt)), N0=n_enroll) / (n_enroll/n)
# the max number gives the total number of pageviews required for this experiemtn
tot_cookies = int(max(N_gc, N_nc, N_rt)*2)
print("Total number of pageviews needed: ", tot_cookies)

Total number of pageviews needed:  6322181


In [8]:
def analytical_size(alpha, beta, dmin, s0):
    nomin = 2*(norm.ppf(1-alpha/2)+norm.ppf(1-beta))**2
    denom = (dmin/s0)**2
    return nomin/denom # per group

Since the change most likey will affect the number of enrollments and thus the amount of payments, there is definite business
risk so that we shouldn't run the experiement on all traffic. We also do not want the experiment to run for too long, but we need at least 14 days to measure net conversion and retention rate. So maybe max duration equals to 30days is constraint.

In [9]:
max_days = 30
max_cookies = max_days * n
print("Maximum number of total cookies can be collected in 30 days: ", max_cookies)

Maximum number of total cookies can be collected in 30 days:  1200000.0


In [10]:
# now if we look at the number of cookies required for each metric
print(2*N_gc, 2*N_nc, 2*N_rt)

856975.0 906075.0 6322181.818181817


Even we run the experiment with 100% traffic for a month, the samples wouldn't be enough for testing the change of retention rate. So let's try relax imposing overall alpha = 0.05 and increasing the practical significance for retention. 

In [11]:
alpha = 0.05
# we require beta = 0.2 for each metric
# the result of each metric needs to be converted to the total number of cookies to be collected
N_gc = test_size(alpha, 0.2, 0.01, np.sqrt(p_gc*(1-p_gc)), N0=n_click) / (n_click/n)
N_nc = test_size(alpha, 0.2, 0.01, np.sqrt(p_nc*(1-p_nc)), N0=n_click) / (n_click/n)
N_rt = test_size(alpha, 0.2, 0.01, np.sqrt(p_rt*(1-p_rt)), N0=n_enroll) / (n_enroll/n)
print(2*N_gc,2*N_nc,2*N_rt)

642475.0 382100.0 4739878.787878788


It's still not enough for the rentation rate. One option is to remove this metric. Since rentation and net conversation rate are quite similar to each other, I think this is acceptable. 

In [12]:
# if we remove retention metric, the percentage traffic we need to run the experiment is 
print(2*max(N_gc, N_nc) / max_cookies)

0.5353958333333333


## Experiement Analysis

### Sanity check

In [13]:
data_control = pd.read_csv("results_control.csv")
data_exp = pd.read_csv("results_experiment.csv")

In [25]:
# we want to merge the data together, so let's add labels for each row
data_control.head()
data_control.tail()

,Date,Pageviews,Clicks,Enrollments,Payments
32,"Wed, Nov 12",10134,801,NaN,NaN
33,"Thu, Nov 13",9717,814,NaN,NaN
34,"Fri, Nov 14",9192,735,NaN,NaN
35,"Sat, Nov 15",8630,743,NaN,NaN
36,"Sun, Nov 16",8970,722,NaN,NaN


In [15]:
data_exp.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


When the metric is a simple count, which should be randomly distributed to either the control or the experiment group, the sample mean should follow normal distribution. If the test of null hypothesis that the distribution is random is not significant, then the sanity check is passed. 

In [16]:
def sanity_test(metric_name, alpha = 0.05):
    tot_control = data_control[metric_name].sum()
    tot_exp = data_exp[metric_name].sum()
    fraction = tot_control / (tot_control + tot_exp)
    std = np.sqrt(0.5**2/(tot_control+tot_exp))
    p = 1- norm.cdf(fraction, 0.5, std)
    
    if p < alpha:
        print("Invariant metric " + metric_name + " failed to pass the sanity test!")
    else:
        print("Invariant metric " + metric_name + " passed the sanity test!")
        
sanity_test("Pageviews")
sanity_test("Clicks")

Invariant metric Pageviews passed the sanity test!
Invariant metric Clicks passed the sanity test!


Click through probability

In [20]:
p_control = data_control["Clicks"].sum() / data_control["Pageviews"].sum() 
p_exp =  data_exp["Clicks"].sum() / data_exp["Pageviews"].sum() 
dif = p_exp - p_control
p_pop = (data_control["Clicks"].sum() + data_exp["Clicks"].sum()) / (data_control["Pageviews"].sum() + data_exp["Pageviews"].sum() )
std = np.sqrt(p_pop*(1-p_pop)*(1/data_exp["Pageviews"].sum() + 1/data_control["Pageviews"].sum()))
alpha = 0.05
p = 1- norm.cdf(dif, 0.5, std)
if p < alpha:
    print("Invariant metric Click-through-probability failed to pass the sanity test!")
else:
    print("Invariant metric Click-through-probability passed the sanity test!")

Invariant metric Click-through-probability passed the sanity test!


### Practical and Statistical Significance

Since the three (or two) evaluation metrics I choose most likely will be positively correlated, the Bonferroni correction might be too conservative. So each evaluation metric, I compute a 95% confidence interval around the difference between two groups.  A metric is statistically significant if the confidence interval does not include 0 and it is practically significant if the confidence interval does not include the practical significance boundary.

Since some data for enrollment and payments are missing, we remove rows with NA values.

In [26]:
data_control.dropna(inplace=True)
data_exp.dropna(inplace=True)

In [43]:
def test(metric_name, nom, denom, alpha=0.05, d_min=0.01):
    p_control = data_control[nom].sum() / data_control[denom].sum() 
    p_exp =  data_exp[nom].sum() / data_exp[denom].sum() 
    dif = p_exp - p_control
    p_pop = (data_control[nom].sum() + data_exp[nom].sum()) / (data_control[denom].sum() + data_exp[denom].sum() )
    std = np.sqrt(p_pop*(1-p_pop)*(1/data_exp[denom].sum() + 1/data_control[denom].sum()))    
    z = norm.ppf(1-alpha/2)
    CI = [dif - z*std, dif + z*std]
    print("---"+ metric_name +"---")
    print("Significant level: ", alpha)
    print("Practical significance boundary: ", d_min)
    print("Oberved difference ", dif)
    print("Confidence inveral ", CI)
    if (0<CI[0] or 0>CI[1]):
        print("This metric is statistical significant.")
    else:
        print("This metric is NOT statistical significant.")
    if (d_min<CI[0] or -d_min>CI[1]):
        print("This metric is practically significant.")
    else:
        print("This metric is NOT practically significant.")    

In [44]:
test("Gross Conversation Rate", "Enrollments", "Clicks")

---Gross Conversation Rate---
Significant level:  0.05
Practical significance boundary:  0.01
Oberved difference  -0.020554874580361565
Confidence inveral  [-0.02912320088750467, -0.011986548273218463]
This metric is statistical significant.
This metric is practically significant.


In [45]:
test("Net Conversation Rate", "Payments", "Clicks", d_min=0.075)

---Net Conversation Rate---
Significant level:  0.05
Practical significance boundary:  0.075
Oberved difference  -0.0048737226745441675
Confidence inveral  [-0.011604500677993734, 0.0018570553289053993]
This metric is NOT statistical significant.
This metric is NOT practically significant.


In [47]:
test("Retention Rate", "Payments", "Enrollments")

---Retention Rate---
Significant level:  0.05
Practical significance boundary:  0.01
Oberved difference  0.031094804707142765
Confidence inveral  [0.008104858181445022, 0.05408475123284051]
This metric is statistical significant.
This metric is NOT practically significant.


### Sign test

Let's run a sign test for our evaluation metrics. It can help to further boost our confidence in our statistical test, if the two tests results agree with each other. If not, sign test can help us to discover hidden segement pattern of our experiement results. 

In [55]:
# let's first calcualte Gross conversion rate and Retention rate for each day for each group
data_control["gcr_c"] = data_control["Enrollments"] / data_control["Clicks"]
data_exp["gcr_e"] = data_exp["Enrollments"] / data_exp["Clicks"]
data_control["rtr_c"] = data_control["Payments"] / data_control["Enrollments"]
data_exp["rtr_e"] = data_exp["Payments"] / data_exp["Enrollments"]

In [59]:
df_merge = data_control.merge(data_exp, on="Date")[["Date", "gcr_c", "rtr_c", "gcr_e", "rtr_e"]]
df_merge.head()

,Date,gcr_c,rtr_c,gcr_e,rtr_e
0,"Sat, Oct 11",0.195051,0.522388,0.153061,0.323810
1,"Sun, Oct 12",0.188703,0.476190,0.147771,0.784483
2,"Mon, Oct 13",0.183718,0.568862,0.164027,0.544828
3,"Tue, Oct 14",0.186603,0.673077,0.166868,0.666667
4,"Wed, Oct 15",0.194743,0.392638,0.168269,0.671429


In [77]:
from scipy.stats import binom
def pv_binom(n, N, p=0.5, alpha=0.05):
    cdf = binom.cdf(n, N, 0.5)
    if cdf > 0.5:
        pv = 1-cdf
    else:
        pv = cdf
    print("p-value ", pv)
    if pv < alpha/2:
        print("The metric passed the sign test!")
    else:
        print("The metric FAILED the sign test!") 


In [78]:
# Gross conversition rate
N = len(df_merge)
n_gc = df_merge[df_merge["gcr_c"]>df_merge["gcr_e"]]["Date"].count()
pv_binom(n_gc, N)

p-value  0.000244140625
The metric passed the sign test!


In [79]:
# retention rate
n_rtr = df_merge[df_merge["rtr_c"]>df_merge["rtr_e"]]["Date"].count()
pv_binom(n_rtr, N)

p-value  0.3388197422027588
The metric FAILED the sign test!


To see why retention rate failed the sign test but passed the stats test, let's study its behaviour for different days.

In [80]:
df_merge["Date"] = df_merge.Date.str.split(',').str[0]
df_merge.head()

,Date,gcr_c,rtr_c,gcr_e,rtr_e
0,Sat,0.195051,0.522388,0.153061,0.323810
1,Sun,0.188703,0.476190,0.147771,0.784483
2,Mon,0.183718,0.568862,0.164027,0.544828
3,Tue,0.186603,0.673077,0.166868,0.666667
4,Wed,0.194743,0.392638,0.168269,0.671429


In [84]:
df_merge["rtr_e>rtr_c"] = df_merge["rtr_e"] > df_merge["rtr_c"]
df_merge.groupby("Date")["rtr_e>rtr_c"].aggregate("mean")

Date
Fri    0.333333
Mon    0.666667
Sat    0.500000
Sun    1.000000
Thu    0.333333
Tue    0.333333
Wed    0.666667
Name: rtr_e>rtr_c, dtype: float64

### Make a recomendation
In summary, we start with three evaluation metrics. We decided to drop one because it requires either impractical amount time or too much traffic to collect enough data to reach required test power. For the leftover metrics, Gross conversion rate change is significant both statistically and practically, while Retention rate change is not significant practically. Therefore, I won't recommend launch this experiment for now. I suggest to dig deeper on the daily pattern of the retention rate change and try to understand why the change only affect vistors on certain days of each week.  

## Follow up experiment